# Turn your Pandas pipelines into data apps with hvplot `.interactive`

In [1]:
# !pip install panel hvplot pandas scipy matplotlib

In [2]:
import pandas as pd
import hvplot.pandas # Adds .hvplot and .interactive methods to Pandas dataframes
import panel as pn # Panel is a simple, flexible and enterprise-ready data app framework

pn.extension(sizing_mode="stretch_width")
pd.set_option("display.precision", 0)

PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]
ACCENT_BASE_COLOR = PALETTE[0]

**load the data ...**

In [3]:
try:
    seattle_bikes = pd.read_csv("seattle_bikes.csv", parse_dates=["Date"]).set_index("Date")
except FileNotFoundError:
    seattle_bikes = pd.read_csv("https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD", parse_dates=["Date"]).set_index("Date")
    seattle_bikes.to_csv("seattle_bikes.csv")

## Define your Pipeline

In [ ]:
def pipeline(seattle_bikes=seattle_bikes, frequency="D", window=50, center=True, win_type="gaussian", std=10):
    return (
        seattle_bikes
        .resample(frequency)
        .sum()
        .rolling(window, center=center, win_type=win_type)
        .sum(std=std)
        .dropna()
    )
pipeline(frequency="W").head(2)

## Define your Widgets!

You can find the widget reference guides [here](https://panel.holoviz.org/reference/index.html#widgets).

In [ ]:
frequency = pn.widgets.Select(value="D", options=["D", "W", "M"], name="Sampling Frequency")
window = pn.widgets.IntSlider(value=50, start=10, end=100, name="Rolling Window Length")
center = pn.widgets.Checkbox(value=True, name="Center")
win_type = pn.widgets.Select(value="gaussian", options=[None, "gaussian"], name="Window Type")
std = pn.widgets.IntSlider(value=10, start=5, end=20, name="std")

## Create an Interactive Pipeline With Widgets!

In [ ]:
ipipeline = pipeline(seattle_bikes.interactive(), frequency=frequency, window=window, center=center, win_type=win_type, std=std)

## Make `.head` Interactive!

In [ ]:
nrows = pn.widgets.IntSlider(value=5, start=1, end=10, name="Rows")
ihead = ipipeline.head(nrows)
ihead

## Make `.plot` Interactive

In [ ]:
line_width = pn.widgets.IntSlider(value=6, start=1, end=10, name="Line Width")
iplot = ipipeline.plot(color=PALETTE, figsize=(15,4), title="Seatle Bike Data with Matplotlib and hvplot .interactive", lw=line_width)
iplot

## Make `.hvplot` Interactive!

`.hvplot` is a Pandas `.plot` drop in replacement that that creates beautiful, interactive plots for the browser.

In [ ]:
ihvplot = ipipeline.hvplot(responsive=True, min_height=400, color=PALETTE, line_width=line_width, yformatter="%.0f", title="Seatle Bike Data with Bokeh and hvplot .interactive")
ihvplot

## Make Anything Interactive with `.pipe`

### Make a Function Interactive

In [ ]:
def message(seattle_bikes, nrows):
    return f"""With a resampling frequency of **{frequency.value}** and window length of **{window.value}**, the aggregated dataframe contains **{len(seattle_bikes)}** rows. Below we show the first **{nrows}** rows."""

imessage = ipipeline.pipe(message, nrows=nrows)
imessage

### Make a Class Interactive

Coming up

## Layout the Data App

In [ ]:
pn.template.FastListTemplate(
    site="", title="Turn your pipelines into data apps with hvplot .interactive and Panel",
    sidebar=[nrows, frequency, window, center, window, std, line_width],
    main=[imessage.panel(), ihead.panel(), iplot.panel(), ihvplot.panel()], 
    accent_base_color=ACCENT_BASE_COLOR,header_background=ACCENT_BASE_COLOR,
).servable();

You can **serve the app** with `panel serve HvplotInteractive.ipynb`. Add `--autoreload` for *hot reloading* while developing. The app is available at [http://localhost:5006/HvplotInteractive](http://localhost:5006/HvplotInteractive).

- For previewing the app in Jupyter lab check out the [Panel Jupyter Lab Preview](https://blog.holoviz.org/panel_0.12.0.html#JupyterLab-previews).
- For deployment options check out the [Server Deployment User Guide](https://panel.holoviz.org/user_guide/Server_Deployment.html).